# Задание
Пакет SURPRISE:

    используйте данные MovieLens 1M,
    можно использовать любые модели из пакета,
    получите RMSE на тестовом сете 0,87 и ниже.



In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

### Загрузка данных

In [ ]:
!wget 'https://drive.google.com/uc?id=1kOh-9U9r0U4e63OAUoB_8JHfrisNxq_-' -O ml-latest-small.zip

--2023-03-06 12:23:44--  https://drive.google.com/uc?id=1kOh-9U9r0U4e63OAUoB_8JHfrisNxq_-
Resolving drive.google.com (drive.google.com)... 142.251.31.139, 142.251.31.113, 142.251.31.102, ...
Connecting to drive.google.com (drive.google.com)|142.251.31.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/m0iciokdb95bvkcj9nkltdo4oeiu2u84/1678105425000/07609373658266146093/*/1kOh-9U9r0U4e63OAUoB_8JHfrisNxq_-?uuid=1ed9ef35-c91a-46c0-bbc9-89c477eb61a7 [following]
--2023-03-06 12:23:45--  https://doc-14-c4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/m0iciokdb95bvkcj9nkltdo4oeiu2u84/1678105425000/07609373658266146093/*/1kOh-9U9r0U4e63OAUoB_8JHfrisNxq_-?uuid=1ed9ef35-c91a-46c0-bbc9-89c477eb61a7
Resolving doc-14-c4-docs.googleusercontent.com (doc-14-c4-docs.googleusercontent.com)... 108.177.119.132, 2a00:1450:4013:c00::84
Connecting to doc-1

In [ ]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
links = pd.read_csv('ml-latest-small/links.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)

In [ ]:
movies_with_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    100836 non-null  int64  
 1   title      100836 non-null  object 
 2   genres     100836 non-null  object 
 3   userId     100836 non-null  int64  
 4   rating     100836 non-null  float64
 5   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 4.6+ MB


In [ ]:
movies_with_ratings

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483
...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,1537109082
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,1537109545
100833,193585,Flint (2017),Drama,184,3.5,1537109805
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021


### Работа с surprise

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 27.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366509 sha256=82b23135f63f6a19b64f73ef24b6479a35314d6cf5aa3ce5b9a461e0a10a7499
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [ ]:
from surprise import (
    accuracy,
    BaselineOnly,
    CoClustering,
    Dataset,
    KNNBaseline,
    KNNBasic,
    KNNWithMeans,
    KNNWithZScore,
    NMF,
    NormalPredictor,
    Reader,
    SlopeOne,
    SVD,
    SVDpp,
)


from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [ ]:
# формирование датасета для обучения
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    # 'mid': movies_with_ratings.movieId,
    'mid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
# data = Dataset.load_from_df(dataset, reader)

In [ ]:
data = Dataset.load_from_df(dataset, reader)

In [ ]:
data

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

In [ ]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': True  # compute  similarities between users
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8985


0.898482528669153

In [ ]:
# Качество RMSE
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8975  0.9041  0.9093  0.8931  0.9023  0.9013  0.0056  
Fit time          0.18    0.20    0.21    0.20    0.22    0.20    0.01    
Test time         2.03    4.55    2.21    1.96    2.23    2.60    0.98    


{'test_rmse': array([0.89748716, 0.9041194 , 0.90933676, 0.89307323, 0.90227912]),
 'fit_time': (0.18465447425842285,
  0.1973409652709961,
  0.21069073677062988,
  0.1991572380065918,
  0.21796560287475586),
 'test_time': (2.025451183319092,
  4.549335718154907,
  2.2139549255371094,
  1.9624085426330566,
  2.2311954498291016)}

#### Выбор алгоритма

In [ ]:
benchmark = []
# перечисление всех алгоритмов
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # вычисление cross_validate
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=False)

    # Добавление результатов в датафрейм
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.869410,68.267226,17.218601
BaselineOnly,0.877419,0.230585,0.335008
SVD,0.880308,1.615539,0.470037
KNNBaseline,0.883481,0.246590,3.475629
KNNWithZScore,0.904679,0.169569,2.963325
KNNWithMeans,0.905327,0.102500,2.845545
SlopeOne,0.910792,3.769159,10.051894
NMF,0.933676,3.462254,0.384162
CoClustering,0.955553,3.490872,0.292466


#### Выбор параметров алгоритма KNNWithMeans

In [ ]:
# подбор оптимальных параметров KNNWithMeans
benchmark = []

models=[
    {'name':'KNNWithMeans_cosine_ub_t','model':KNNWithMeans(k=50, sim_options={'name': 'cosine', 'user_based': True})},
    {'name':'KNNWithMeans_cosine_ub_f','model':KNNWithMeans(k=50, sim_options={'name': 'cosine', 'user_based': False})},
    {'name':'KNNWithMeans_msd_ub_t','model':KNNWithMeans(k=50, sim_options={'name': 'msd', 'user_based': True})},
    {'name':'KNNWithMeans_msd_ub_f','model':KNNWithMeans(k=50, sim_options={'name': 'msd', 'user_based': False})},
    {'name':'KNNWithMeans_pearson_ub_t','model':KNNWithMeans(k=50, sim_options={'name': 'pearson', 'user_based': True})},
    {'name':'KNNWithMeans_pearson_ub_f','model':KNNWithMeans(k=50, sim_options={'name': 'pearson', 'user_based': False})}
]

for algorithm in models:
    # вычисление cross_validate
    results = cross_validate(algorithm['model'], data, measures=['RMSE'], cv=5, verbose=False)
    # Добавление результатов в датафрейм
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    # tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    tmp = tmp.append(pd.Series(algorithm['name'], index=['Algorithm']))
    benchmark.append(tmp)

pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd sim

,test_rmse,fit_time,test_time
Algorithm,,,
KNNWithMeans_msd_ub_f,0.890791,3.593358,12.234503
KNNWithMeans_msd_ub_t,0.896049,0.125238,1.925652
KNNWithMeans_pearson_ub_t,0.897702,0.212768,1.976670
KNNWithMeans_pearson_ub_f,0.901300,6.469630,12.121142
KNNWithMeans_cosine_ub_t,0.901627,0.168772,2.058931
KNNWithMeans_cosine_ub_f,0.901958,5.763727,12.219026


## **Вывод** - SVDpp  показывает лучший результат (0.869410)